# Imports

In [ ]:
# !pip install yfinance
# !pip install pmdarima
# !pip install hyperopt
# !pip install xgboost
# !pip install numpy -U

In [1]:
import os
from functools import partial
from datetime import datetime

import typing
from typing import Dict
from typing import Any
from typing import Tuple

import numpy as np
import pandas as pd
import sklearn

import utils
import pipelines
import processing
import evaluate
import predict
import preprocessing

# Download stock daily prices & indexes

In [14]:
### Gets all ticker names (no argument given)
ticker_list = utils.get_ticker_names(market_cap_min_mm=1000, market_cap_max_mm=None)

In [15]:
### Specific date - 3rd of March 2022 (Y, M, D)
# date_to = datetime(2021, 1, 18)
### Date of today
date_to = datetime.today()
### How many years' of data to download (going backwards from date_end). Year can be a floating point number
period_years = 8

In [16]:
df, df_clean = utils.download_stonk_prices(
    ticker_list.index, period_years=period_years, date_to=date_to
)
vix, vix_clean = utils.download_stonk_prices(
    ["^VIX"], period_years=period_years, date_to=date_to, fname_prefix="vix"
)
sp500, sp500_clean = utils.download_stonk_prices(
    ["^GSPC"], period_years=period_years, date_to=date_to, fname_prefix="sp500"
)

[*********************100%***********************]  2820 of 2820 completed

16 Failed downloads:
- DELL WI: No data found, symbol may be delisted
- BLL: No data found, symbol may be delisted
- RXN WI: No data found, symbol may be delisted
- O.WI: No data found, symbol may be delisted
- SNX.WI: No data found, symbol may be delisted
- FOE: No data found, symbol may be delisted
- SGMS: No data found, symbol may be delisted
- BIP.PRB: No data found, symbol may be delisted
- T WD: No data found, symbol may be delisted
- BIP.PRA: No data found, symbol may be delisted
- PFE.WI: No data found, symbol may be delisted
- EPAY: No data found, symbol may be delisted
- JOBS: No data found, symbol may be delisted
- MRK.WI: No data found, symbol may be delisted
- MGP: No data found, symbol may be delisted
- POST WI: No data found, symbol may be delisted
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


# Run data pipeline

In [2]:
industries = [
    # 'health_care_equipment_and_services',
    # 'software_and_services',
    # 'retailing',
    # 'telecommunication_services',
    "capital_goods",
    # 'energy',
    # 'pharmaceuticals_biotechnology_and_life_sciences',
    # 'consumer_staples',
    # 'banks',
    # 'diversified_financials',
    # 'metals_and_mining',
    # 'technology_hardware_and_equipment',
    # 'utilities',
    # 'chemicals',
    # 'automobiles_and_components',
    # 'semiconductors_and_semiconductor_equipment',
    # 'media_and_entertainment',
    # 'real_estate',
    # 'consumer_services',
    # 'consumer_durables_and_apparel',
    # 'insurance',
    # 'transportation',
    # 'commercial_and_professional_services',
    # 'paper_and_forest_products',
    # 'containers_and_packaging',
    # 'construction_materials'
]

l_reg = 3
l_roll = 2
dt = 10

output_dir = "data"

stonk_model = predict.XGBStonkModel()
vix = utils.get_stonk_data(fname_prefix="vix", disable_filter=True).iloc[0]

In [3]:
datasets = []
i = 1
total_industries = len(industries)
for industry in industries:
    stonks = utils.get_stonk_data(filter_industries=[industry])
    X, Y = processing.combine_stonk_pairs(stonks)

    print("Industry ({0}/{1}): {2}".format(i, total_industries, industry))

    print("Processing residuals...")
    residuals, betas, _, date_index = utils.measure_time(
        partial(
            processing.get_rolling_residuals,
            X=X,
            Y=Y,
            l_reg=l_reg,
            l_roll=l_roll,
            dt=dt,
        )
    )

    std_residuals, means, stds = processing.get_standardized_residuals(
        residuals
    )

    trades_before = len(std_residuals)
    std_residuals = std_residuals[std_residuals.iloc[:, -1].abs() >= 2.5]
    trades_after = len(std_residuals)
    print(
        "{0} trades selected out of {1} by residual values".format(
            trades_after, trades_before
        )
    )
    if trades_after == 0:
        continue
        
    residuals = residuals.loc[std_residuals.index]
    betas = betas.loc[std_residuals.index]
    intercepts = intercepts.loc[std_residuals.index]
    dates_index = dates_index.loc[std_residuals.index]

    print("Processing ADFs...")
    adfs, adfs_raw = utils.measure_time(
        partial(
            processing.get_aggregate_adfs,
            residuals,
            betas=betas,
        )
    )

    selected_by_adf = (adfs >= 0.5).values
    adfs = adfs[selected_by_adf]

    trades_before = len(std_residuals)
    std_residuals = std_residuals[selected_by_adf]
    trades_after = len(std_residuals)
    print(
        "{0} trades selected out of {1} by ADF pass rates".format(
            trades_after, trades_before
        )
    )

    if len(std_residuals) == 0:
        continue

    betas = betas.loc[adfs.index]
    betas_last = betas_last.loc[std_residuals.index]
    
    residuals = residuals.loc[adfs.index]
    adfs_raw = adfs_raw.loc[adfs.index]
    
    dates_index = dates_index.loc[std_residuals.index]
    
    means = means.loc[std_residuals.index]
    stds = stds.loc[std_residuals.index]

    residuals_max_mean = processing.get_mean_residual_magnitude(
        std_residuals.to_numpy(), dt=21
    )
    print(
        "Mean max residual value for {0} after filtering is {1}".format(
            industry, residuals_max_mean
        )
    )
    
    print("Processing beta stability tests...")
    beta_stability_rsquared_vals = utils.measure_time(
        partial(
            processing.calculate_beta_stability_rsquared,
            prices_X=X_until_T, prices_Y=Y_until_T, betas=betas, dates_index=dates_index
        )
    )
    assert np.all(beta_stability_rsquared_vals.index == std_residuals.index)
    
    print("Processing ARIMA forecasts...")
    arima_forecasts = utils.measure_time(
        partial(
            processing.calculate_arima_forecast,
            std_residuals=std_residuals,
            forecast_months=arima_forecast_months,
            eval_models=arima_eval_models,
        )
    )

    print("Preparing data for model...")
    dataset = utils.build_dataset_from_live_data_by_industry(
        std_residuals=std_residuals,
        adfs=adfs,
        subindustry=industry,
        mean_max_residual=residuals_max_mean,
        vix_index=vix.loc[stonks.columns[-1]],
        betas_stability_rsquared=beta_stability_rsquared_vals
        arima_forecasts=arima_forecasts
    )

    print("Running model...")
    predictions, df_processed = stonk_model.predict(dataset)
    datasets.append((dataset, df_processed))
    predictions = pd.DataFrame(predictions)
    predictions.index = adfs.index

    residuals.insert(0, "dates", date_index)
    betas.insert(0, "dates", date_index)
    
    print("Writing results to CSV...")
    # Very big industry, exceeds Git file size limit
    if industry == "diversified_financials":
        half = len(residuals) // 2
        residuals_fst = residuals.iloc[:half]
        residuals_snd = residuals.iloc[half:]
        residuals_fst.to_csv(
            os.path.join(output_dir, industry + "_one_residuals.csv"),
            header=False,
            index=True,
        )
        residuals_snd.to_csv(
            os.path.join(output_dir, industry + "_two_residuals.csv"),
            header=False,
            index=True,
        )
        del residuals_fst
        del residuals_snd
    else:
        residuals.to_csv(
            os.path.join(output_dir, industry + "_residuals.csv"),
            header=False,
            index=True,
        )
    betas.to_csv(
        os.path.join(output_dir, industry + "_betas.csv"), header=False, index=True
    )
    adfs_raw.to_csv(
        os.path.join(output_dir, industry + "_adfs_raw.csv"), header=False, index=True
    )
    predictions.to_csv(
        os.path.join(output_dir, industry + "_predictions.csv"),
        header=False,
        index=True,
    )
    i += 1

print("*** All done ***")

Industry (1/1): capital_goods
Processing residuals...
Done after: 33s
807 trades selected out of 16836 by residual values
Processing ADFs...
Done after: 80s
166 trades selected out of 807 by ADF pass rates
Mean max residual value for capital_goods after filtering is 3.869999885559082
Preparing data for model...
Running model...
Writing results to CSV...
*** All done ***


# Data collection

In [2]:
stonks = utils.get_stonk_data()
stonks = stonks.loc[:, :"2020-05-08"]

In [ ]:
pipelines.data_collection_rolling_pipeline(
    stonks,
    l_reg=3,
    l_roll=2,
    dt=10,
    market_cap_min_mm=1000,
    market_cap_max_mm=None,
    last_residual_cutoff=2.5,
    mean_max_residual_dt=21,
    adf_pval_cutoff=0.1,
    adf_pass_rate_filter=0.5,
    arima_forecast_months=3,
    arima_eval_models=5,
    trade_length_months=3,
    trading_interval_weeks=2,
    # first_n_windows=72,
)

Total data windows: 15


In [8]:
dataset = utils.ingest_trade_pipeline_outputs()

vix = utils.get_stonk_data(fname_prefix="vix", disable_filter=True).iloc[0]
sp500 = utils.get_stonk_data(fname_prefix="sp500", disable_filter=True).iloc[0]

sp500_chg = pd.Series((sp500.iloc[63:].values / sp500.iloc[:-63].values) - 1)
sp500_chg.index = sp500.iloc[63:].index

dataset["vix"] = dataset["trade_date"].apply(lambda x: vix.loc[x])
dataset["sp500"] = dataset["trade_date"].apply(lambda x: sp500_chg.loc[x])
dataset.to_csv("data/dataset.csv", header=True, index=False)

# Model development

In [2]:
import xgboost as xgb
from hyperopt import STATUS_OK, STATUS_FAIL, Trials, fmin, hp, tpe, atpe, rand
import pickle

In [ ]:
def train_production_xgb(
    df: pd.DataFrame, params: Dict[str, Any], noise_level: float = 0
) -> Tuple[xgb.XGBClassifier, sklearn.base.TransformerMixin]:
    X_train, scalers = preprocessing.transform_features(df, noise_level=noise_level)
    y_train = df["label"]

    clf = xgb.XGBClassifier(**params)

    clf.fit(X_train, y_train, eval_set=[(X_train, y_train)])
    clf.save_model(os.path.join("data", "xgb_classifier.json"))

    with open(os.path.join("data", "scalers.json"), "wb") as fp:
        pickle.dump(scalers, fp)

    return clf, scalers

In [3]:
df = pd.read_csv("data/dataset.csv")
df = df[df.beta > 0]
df = df[df.last_residual.abs() >= 2.5]
df = preprocessing.assign_labels(df)

In [ ]:
drop_dates = 2
selected_dates = np.sort(df["trade_date"].unique())[drop_dates:]
df_prod = df[df.trade_date.isin(selected_dates)].sample(frac=1)
print(len(df_prod))
print(df_prod["label"].value_counts())

In [ ]:
clf_prod, scalers_prod = train_production_xgb(df_prod, params, noise_level=0.005)

In [38]:
splits = preprocessing.split_data(df, 2, 6, 0, random_state=343439)
print(len(splits["train"]))
print(len(splits["validation"]))
print(splits["train"]["label"].value_counts())
print(splits["validation"]["label"].value_counts())

91869
3605
0    79130
1    12739
Name: label, dtype: int64
0    2966
1     639
Name: label, dtype: int64


In [69]:
noise_level = 0.005

X_train, scalers = preprocessing.transform_features(
    splits["train"], noise_level=noise_level
)
X_valid, _ = preprocessing.transform_features(
    splits["validation"], scalers=scalers, noise_level=0
)

y_train = splits["train"]["label"]
y_valid = splits["validation"]["label"]

In [36]:
hyperparameter_space = {
    "gamma": hp.uniform("gamma", 0, 5),
    "scale_pos_weight": hp.uniform("scale_pos_weight", 2, 12),
    "max_depth": hp.quniform("max_depth", 3, 12, 1),
    "min_child_weight": hp.quniform("min_child_weight", 1, 8, 1),
    "max_delta_step": hp.quniform("max_delta_step", 1, 4, 1),
    "n_estimators": hp.quniform("n_estimators", 50, 150, 1),
    # "n_estimators": hp.choice("n_estimators", np.array([50, 75, 100, 150, 200])),
    # "subsample": hp.uniform("subsample", 0.5, 1),
    # "colsample_bylevel" : hp.uniform("colsample_bylevel", 0.5, 1),
}

In [37]:
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score


def optimization_objective(space):
    clf = xgb.XGBClassifier(
        gamma=space["gamma"],
        scale_pos_weight=space["scale_pos_weight"],
        #
        max_depth=int(space["max_depth"]),
        min_child_weight=int(space["min_child_weight"]),
        max_delta_step=int(space["max_delta_step"]),
        #
        # colsample_bylevel = space['colsample_bylevel'],
        colsample_bylevel=1,
        n_estimators=int(space["n_estimators"]),
        learning_rate=0.1,
        # subsample = space['subsample'],
        subsample=1,
        #
        tree_method="hist",
        enable_categorical=True,
        max_cat_to_onehot=1,
        random_state=np.random.randint(9999999),
    )

    clf.fit(
        X_train,
        y_train,
        verbose=False,
    )

    y_score = clf.predict_proba(X_valid)[:, 1]
    y_preds = y_score > 0.5

    f1 = f1_score(y_valid, y_preds, zero_division=0)
    precision = precision_score(y_valid, y_preds, zero_division=0)
    ap = evaluate.average_precision_from_cutoff(y_valid, y_score, 0.4)
    roc = roc_auc_score(y_valid, y_score)

    pos_preds = int(y_preds.sum())
    pos_labels = int(y_valid.sum())

    ap = ap if pos_preds >= pos_labels else 0

    if f1 == 0 or precision == 0:
        return {
            "loss": 100,
            "precision": precision,
            "f1_score": f1,
            "ap": ap,
            "auc": roc,
            "pos_preds": pos_preds,
            "status": STATUS_FAIL,
        }
    else:
        return {
            "loss": -ap,
            "precision": precision,
            "f1_score": f1,
            "ap": ap,
            "auc": roc,
            "pos_preds": pos_preds,
            "status": STATUS_OK,
        }

In [71]:
trials = Trials()

best_hyperparams = fmin(
    fn=optimization_objective,
    space=hyperparameter_space,
    algo=tpe.suggest,
    max_evals=500,
    trials=trials,
)

trial_vals = trials.vals
trial_vals["f1_score"] = list(map(lambda x: x["f1_score"], trials.results))
trial_vals["precision"] = list(map(lambda x: x["precision"], trials.results))
trial_vals["ap"] = list(map(lambda x: x["ap"], trials.results))
trial_vals["auc"] = list(map(lambda x: x["auc"], trials.results))
trial_vals["pos_preds"] = list(map(lambda x: x["pos_preds"], trials.results))

df_trials = pd.DataFrame.from_dict(trial_vals)
df_trials.to_csv("data/experiments/new_dataset_test_no-betas#1.csv", index=False)

100%|██████████| 500/500 [03:57<00:00,  2.11trial/s, best loss: -0.2870183688569616]


In [42]:
params = {
    # reg def 0
    "gamma": 3.965115,
    # L2 def 1
    # "reg_lambda" : 1,
    # "reg_alpha" : 0,
    # Class imbalance def 1
    "scale_pos_weight": 4.635863,
    # Integers:
    "max_depth": 6,
    # Reg def 1
    "min_child_weight": 6,
    # Class imbalance def 0
    "max_delta_step": 4,
    # Choice:
    "colsample_bylevel": 1,
    "n_estimators": 69,
    "learning_rate": 0.1,
    "subsample": 1,
    # Fixed:
    "tree_method": "hist",
    "enable_categorical": True,
    "max_cat_to_onehot": 1,
    "eval_metric": ["logloss"],
    "random_state": np.random.randint(999929),
}

clf = xgb.XGBClassifier(**params)

clf.fit(X_train, y_train, eval_set=[(X_valid, y_valid), (X_train, y_train)])

[0]	validation_0-logloss:0.67008	validation_1-logloss:0.66562
[1]	validation_0-logloss:0.65062	validation_1-logloss:0.64257
[2]	validation_0-logloss:0.63404	validation_1-logloss:0.62309
[3]	validation_0-logloss:0.62021	validation_1-logloss:0.60662
[4]	validation_0-logloss:0.60835	validation_1-logloss:0.59248
[5]	validation_0-logloss:0.59877	validation_1-logloss:0.58025
[6]	validation_0-logloss:0.59006	validation_1-logloss:0.56944
[7]	validation_0-logloss:0.58237	validation_1-logloss:0.55938
[8]	validation_0-logloss:0.57583	validation_1-logloss:0.55099
[9]	validation_0-logloss:0.56930	validation_1-logloss:0.54324
[10]	validation_0-logloss:0.56320	validation_1-logloss:0.53625
[11]	validation_0-logloss:0.55897	validation_1-logloss:0.53041
[12]	validation_0-logloss:0.55468	validation_1-logloss:0.52478
[13]	validation_0-logloss:0.55181	validation_1-logloss:0.51892
[14]	validation_0-logloss:0.55203	validation_1-logloss:0.51366
[15]	validation_0-logloss:0.55135	validation_1-logloss:0.50904
[1

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=True,
              eval_metric=['logloss'], gamma=3.965115, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.1, max_bin=256,
              max_cat_to_onehot=1, max_delta_step=4, max_depth=6, max_leaves=0,
              min_child_weight=6, missing=nan, monotone_constraints='()',
              n_estimators=69, n_jobs=0, num_parallel_tree=1, predictor='auto',
              random_state=830831, reg_alpha=0, reg_lambda=1, ...)

In [59]:
print("**Validation**")
y_score = clf.predict_proba(X_valid)[:, 1]
thres = 0.5
y_preds = y_score > thres

evaluate.performance_summary(y_score, y_preds, y_valid, auc_cutoff=0.4)

df_results_valid = evaluate.returns_on_predictions(splits["validation"], y_preds)

evaluate.performance_on_slice(
    splits["validation"], y_score, y_preds, "subindustry", False
)

**Validation**
Precision: 0.2694541231126597
PR-AUC/AP score: 0.302656177366849
ROC-AUC score: 0.5868035439730614
Total positive predictions: 861

Totals:
        prediction
result            
FN             407
FP             629
TN            2337
TP             232

Means:
        return_one_month  return_two_month  return_three_month
result                                                        
FN              0.070713          0.114047            0.125668
FP              0.000339          0.001690           -0.003358
TN              0.000157         -0.000342            0.012303
TP              0.079267          0.145233            0.143914

Stds:
        return_one_month  return_two_month  return_three_month
result                                                        
FN              0.051007          0.066938            0.071777
FP              0.051835          0.059907            0.071096
TN              0.046194          0.062043            0.060137
TP              0.05880

/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_ranking.py:864: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_ranking.py:864: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]


In [61]:
pd.set_option("display.max_rows", 100)

In [ ]:
df_results_valid[df_results_valid.result == "TP"].iloc[:100]

In [ ]:
# df_results_valid[df_results_valid.subindustry == 'consumer_services'].iloc[0:100]

In [64]:
for name, importance in zip(clf.feature_names_in_, clf.feature_importances_):
    print(name, importance)

adf_pass_rate 0.04536544
last_residual 0.091166325
residual_mean_max 0.10575237
vix 0.33035636
betas_rsquared 0.05880348
arima_forecast 0.121871546
industry 0.19851254
residual_inter 0.04817193


In [41]:
df_trials = pd.read_csv("data/experiments/new_dataset_test#3.csv")
df_trials.sort_values("ap", ascending=False).head(50)

,gamma,max_delta_step,max_depth,min_child_weight,n_estimators,scale_pos_weight,f1_score,precision,ap,auc,pos_preds
416,3.926215,4.0,6.0,7.0,69.0,4.299799,0.302242,0.280914,0.306593,0.596298,744
419,3.965115,4.0,6.0,6.0,69.0,4.635863,0.309333,0.269454,0.302656,0.586804,861
403,4.439001,4.0,6.0,7.0,71.0,4.406618,0.303725,0.280053,0.301877,0.586417,757
365,4.930385,2.0,6.0,7.0,76.0,4.545162,0.313484,0.278589,0.298053,0.589931,822
421,3.636291,4.0,6.0,5.0,59.0,4.623494,0.310935,0.268487,0.297951,0.592459,879
356,4.539174,3.0,6.0,7.0,78.0,4.270644,0.298802,0.271795,0.297642,0.593060,780
435,3.598246,4.0,7.0,5.0,64.0,4.018579,0.293803,0.287425,0.297205,0.595116,668
373,4.489548,4.0,5.0,7.0,85.0,4.496963,0.297258,0.275770,0.296872,0.587962,747
392,4.482191,4.0,5.0,7.0,84.0,4.914434,0.305038,0.272840,0.293825,0.591753,810
304,1.651246,2.0,4.0,8.0,125.0,4.317316,0.270769,0.266263,0.292573,0.587240,661


In [68]:
df_trials = pd.read_csv("data/experiments/new_dataset_test_no-arima#1.csv")
df_trials.sort_values("ap", ascending=False).head(50)

,gamma,max_delta_step,max_depth,min_child_weight,n_estimators,scale_pos_weight,f1_score,precision,ap,auc,pos_preds
262,4.194326,2.0,3.0,1.0,100.0,5.226743,0.235378,0.216252,0.269614,0.548180,763
189,2.196664,3.0,3.0,4.0,139.0,5.403251,0.250000,0.206755,0.268622,0.549491,977
406,3.099042,4.0,3.0,1.0,147.0,5.396556,0.243413,0.203141,0.266143,0.542417,955
405,3.125661,4.0,3.0,1.0,145.0,5.382396,0.251250,0.209157,0.265281,0.547447,961
79,3.119032,4.0,3.0,1.0,108.0,4.952060,0.223765,0.220700,0.263730,0.546561,657
407,2.862711,4.0,3.0,1.0,147.0,5.772659,0.261638,0.209830,0.263388,0.545938,1058
186,2.935397,3.0,3.0,4.0,139.0,5.187893,0.244652,0.213536,0.262205,0.544392,857
271,3.733772,1.0,3.0,1.0,94.0,5.524299,0.238128,0.212531,0.261339,0.547929,814
156,3.343460,4.0,3.0,1.0,142.0,5.033297,0.244233,0.215569,0.260774,0.548020,835
257,4.349718,2.0,3.0,2.0,102.0,5.358674,0.230186,0.205665,0.260758,0.545197,812


In [ ]:
df_trials = pd.read_csv("data/data-window-size-2#6.csv")
df_trials.sort_values("ap", ascending=False).head(50)